In [8]:
import sys
print sys.executable

paths = ['', '/Users/dyawitz/anaconda/bin', '/Users/dyawitz/anaconda/lib/python27.zip', '/Users/dyawitz/anaconda/lib/python2.7/plat-darwin', '/Users/dyawitz/anaconda/lib/python2.7/plat-mac', '/Users/dyawitz/anaconda/lib/python2.7/plat-mac/lib-scriptpackages', '/Users/dyawitz/anaconda/lib/python2.7/lib-tk', '/Users/dyawitz/anaconda/lib/python2.7/lib-old', '/Users/dyawitz/anaconda/lib/python2.7/lib-dynload', '/Users/dyawitz/anaconda/lib/python2.7/site-packages/Sphinx-1.3.1-py2.7.egg', '/Users/dyawitz/anaconda/lib/python2.7/site-packages/setuptools-18.5-py2.7.egg', '/Users/dyawitz/anaconda/lib/python2.7/site-packages', '/Users/dyawitz/anaconda/lib/python2.7/site-packages/aeosa', '/Users/dyawitz/anaconda/lib/python2.7/site-packages/IPython/extensions', '/Users/dyawitz/.ipython']

for i in paths:
   sys.path.append(i)

/Users/dyawitz/anaconda/envs/py27/bin/python


In [4]:
import zipfile
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import urllib


Data from the NYS ISO.

See: http://www.nyiso.com/public/markets_operations/market_data/load_data/index.jsp

Directly from `http://mis.nyiso.com/public/P-58Blist.htm`

In [88]:
dates = pd.date_range(pd.to_datetime('2001-01-01'), \
                       pd.to_datetime('2015-12-31'), freq='M')


'20111201pal_csv'

In [103]:
for date in dates:
    url = 'http://mis.nyiso.com/public/csv/pal/{0}{1}01pal_csv.zip'.format(date.year, str(date.month).zfill(2))
    urllib.urlretrieve(url, "../data/nyiso/{0}".format(url.split('/')[-1]))

Download all your zip files to the `../data/nyiso` folder. 

Then, extract every csv into the `../data/nyiso/all/raw` folder

In [104]:
def unzip(source_filename, dest_dir):
    with zipfile.ZipFile(source_filename) as zf:
        zf.extractall(dest_dir)

In [105]:
import os
import pandas as pd
import numpy as np

In [109]:
zips = []
for file in os.listdir("../data/nyiso"):
    if file.endswith(".zip"):
        zips.append(file)

In [111]:
for z in zips:
    unzip('../data/nyiso/' + z, '../data/nyiso/all/raw')

Combine all csvs into one file: combined.csv

In [112]:
csvs = []
for file in os.listdir("../data/nyiso/all/raw"):
    if file.endswith("pal.csv"):
        csvs.append(file)

In [114]:
fout=open("../data/nyiso/all/combined.csv","a")

# write the entire first file:
for line in open("../data/nyiso/all/raw/"+csvs[0]):
    fout.write(line)
# now the rest, skipping the headers:    
for file in csvs[1:]:
    f = open("../data/nyiso/all/raw/"+file)
    f.next() # skip the header
    for line in f:
         fout.write(line)
    f.close() # not really needed
fout.close()

In [115]:
df = pd.read_csv("../data/nyiso/all/combined.csv")

In [116]:
cols = df.columns

In [117]:
df.columns = [col.lower().replace(' ', '') for col in cols]

In [ ]:
#clean out any headers that may be in the data and take the subset of these 4 columns
df = df[['timestamp', 'name', 'ptid', 'load']][df.load != 'Load']

In [136]:
df.to_csv('../data/nyiso/all/combined.csv', index=False)

In [138]:
df.name.unique()

array(['CAPITL', 'CENTRL', 'DUNWOD', 'GENESE', 'HUD VL', 'MHK VL',
       'MILLWD', 'N.Y.C._LONGIL', 'NORTH', 'WEST', 'LONGIL', 'N.Y.C.'], dtype=object)

In [45]:
df = pd.read_csv('../data/nyiso/all/combined.csv')

In [60]:
regions = list(df.name.unique())
region_names = ['Capital', 'Central', 'Dunwoodie', 'Genese', 'Hudson Valley', 'Long Island', 'Mohawk Valley', 'Millwood', 'NYC', 'North', 'West']
cities = ['Albany', 'Syracuse', 'Yonkers', 'Rochester', 'Poughkeepsie', 'NYC', 'Utica', 'Yonkers', 'NYC', 'Plattsburgh', 'Buffalo']
weather_stations = ['kalb', 'ksyr', 'klga', 'kroc', 'kpou', 'kjfk', 'krme', 'klga', 'kjfk', 'kpbg', 'kbuf']

In [61]:
weather_dict = dict(zip(regions, zip(weather_stations, region_names, cities)))

In [64]:
weather_dict

{'CAPITL': ('kalb', 'Capital', 'Albany'),
 'CENTRL': ('ksyr', 'Central', 'Syracuse'),
 'DUNWOD': ('klga', 'Dunwoodie', 'Yonkers'),
 'GENESE': ('kroc', 'Genese', 'Rochester'),
 'HUD VL': ('kpou', 'Hudson Valley', 'Poughkeepsie'),
 'LONGIL': ('kjfk', 'Long Island', 'NYC'),
 'MHK VL': ('krme', 'Mohawk Valley', 'Utica'),
 'MILLWD': ('klga', 'Millwood', 'Yonkers'),
 'N.Y.C.': ('kjfk', 'NYC', 'NYC'),
 'NORTH': ('kpbg', 'North', 'Plattsburgh'),
 'WEST': ('kbuf', 'West', 'Buffalo')}

In [63]:
import joblib
joblib.dump(weather_dict, 'weather_dict.pkl')

['weather_dict.pkl']

In [67]:
weather_dict['CAPITL'][1].lower()

'capital'

In [75]:
sorted(weather_dict.keys())

['CAPITL',
 'CENTRL',
 'DUNWOD',
 'GENESE',
 'HUD VL',
 'LONGIL',
 'MHK VL',
 'MILLWD',
 'N.Y.C.',
 'NORTH',
 'WEST']

In [76]:
region = 'DUNWOD'
subset = df[df.name == region].copy()
filename = weather_dict[region][1].lower().replace(' ', '') + '.csv'
subset.to_csv('../data/nyiso/all/' + filename)

In [77]:
for region in weather_dict.keys():
    subset = df[df.name == region].copy()
    filename = weather_dict[region][1].lower().replace(' ', '') + '.csv'
    subset.to_csv('../data/nyiso/all/' + filename, index=False)

Take an extract of the data for the "CAPITL" region only. This is the area around Albany. A smaller spaital resoluiton will make tying in weather data more accurate.

Plus, it makes the file easier to work with. 

In [139]:
capital = df[df.name == 'CAPITL']

In [140]:
len(capital)

1691882

In [141]:
capital.to_csv('../data/nyiso/all/capital_region.csv', index=False)

In [102]:
capital = pd.read_csv('../data/nyiso/all/capital_region.csv')

In [126]:
capital.timestamp = capital.timestamp.apply(lambda x: pd.to_datetime(x))

In [154]:
capital.timestamp.year

2013

In [134]:
type(capital.load[440823])

numpy.float64

In [172]:
capital.head()

,timestamp,name,ptid,load
0,2012-01-01 00:00:00,CAPITL,61757,1084.4
1,2012-01-01 00:05:00,CAPITL,61757,1055.3
2,2012-01-01 00:10:00,CAPITL,61757,1056.6
3,2012-01-01 00:15:00,CAPITL,61757,1050.8
4,2012-01-01 00:20:00,CAPITL,61757,1050.8


# Output 2012 data to test on

In [158]:
capital[capital.timestamp < pd.to_datetime('2013-01-01')].to_csv('load2012.csv', index=False)

In [160]:
csvs = []
for file in os.listdir("../data/wunderground/kalb"):
    if file.startswith("2012"):
        csvs.append(file)

fout=open("weather2012.csv","a")

# write the entire first file:
for line in open("../data/wunderground/kalb/"+csvs[0]):
    fout.write(line)
# now the rest, skipping the headers:    
for file in csvs[1:]:
    f = open("../data/wunderground/kalb/"+file)
    f.next() # skip the header
    for line in f:
         fout.write(line)
    f.close() # not really needed
fout.close()

# Download historical forecasts
Use these to compare our forecasts to the NYISO

Use the day-ahead forecasts from the NYISO website: http://www.nyiso.com/public/markets_operations/market_data/custom_report/index.jsp?report=load_forecast

In [6]:
nyiso_forecast = pd.read_csv('../data/nyiso_dayahead_forecasts/forecast_2014_2016.csv')

In [7]:
len(nyiso_forecast)

211442

In [11]:
nyiso_forecast.columns = ['timestamp', 'zone', 'forecast', 'gmt']


In [13]:
capital_forecast = nyiso_forecast[nyiso_forecast.zone == 'CAPITL']

In [17]:
capital_forecast.reset_index(inplace=True)

In [19]:
capital_forecast = capital_forecast[['timestamp', 'zone', 'forecast']]

In [21]:
capital_forecast.to_csv('../data/nyiso_dayahead_forecasts/capital_forecast.csv', index=False)

In [3]:
forecast_48 = [46,47,47,46,45,43,42,41,41,40,39,38,37,36,36,37,38,40,41,43,44,44,45,45,45,43,41,38,36,34,32,31,30,28,27,26,26,25,25,28,34,39,43,47,51,54,55,56,56,52,49,47,46,45,45,44,42,41,40,39,39,39,39,42,46,50,54,57,60,62,63,64]

In [6]:
phour = pd.to_datetime('1 hour')
start_time = pd.to_datetime('2016-03-28')
[range(1,73)]

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72]